In [1]:
import semantic_kernel as sk
import requests

from semantic_kernel.skill_definition import sk_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.planning.basic_planner import BasicPlanner
from dotenv import load_dotenv, find_dotenv

class UnitedStatesPlugin:
    @sk_function(
        description="Get the United States population for a specific year",
        name="GetPopulation",
        input_description="The year to get the population for"
    )

    def get_population(self, input: str) -> str:
        url = "https://datausa.io/api/data?drilldowns=Nation&measures=Population"
        response = requests.get(url)
        data = response.json()
        for record in data["data"]:
            if record["Year"] == input:
                return "The population number in the United States in {year} was {population}".format(year=input, population=record["Population"])
        return "No data found for that year"
    


kernel = sk.Kernel()

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
kernel.add_chat_service("dv", AzureChatCompletion(deployment, endpoint, api_key))

skills_directory = "../Plugins/"
kernel.import_semantic_skill_from_directory(skills_directory, "MailPlugin")
kernel.import_skill(UnitedStatesPlugin(), skill_name="UnitedStatesPlugin")

ask = """
Write a mail to share the number of the United States population in 2015 for a research program.
"""

planner = BasicPlanner()
basic_plan = await planner.create_plan_async(ask, kernel)

print(basic_plan.generated_plan)

results = await planner.execute_plan_async(basic_plan, kernel)
print(results)

{
        "input": "US population in 2015",
        "subtasks": [
            {"function": "UnitedStatesPlugin.GetPopulation", "args": {"input": 2015}},
            {"function": "MailPlugin.WriteBusinessMail", "args": {"input": "Please find attached the number of US population in 2015 for the research program."}}
        ]
    }
Dear [Recipient],

I hope this email finds you well. I am writing to provide you with the number of US population in 2015, which is attached to this email. This information is intended for use in the research program.

Please let me know if you require any further assistance or information. I am always here to help.

Best regards,

AI Assistant
